# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

The Primary Key for the song_events table is session_id as the partition key and item_in_session as the clustering key. This allows the query to use these columns as filters.

In [8]:
query = "CREATE TABLE IF NOT EXISTS song_events "
query = query + '''(session_id int,
                    item_in_session int,
                    artist text,
                    song text,
                    length float,
                    PRIMARY KEY ((session_id), item_in_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [9]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO song_events (session_id,
                                       item_in_session,
                                       artist,
                                       song,
                                       length)'''
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), str(line[0]), str(line[9]), float(line[5])))

In [10]:
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query_1 = '''SELECT artist, song, length
           FROM song_events
           WHERE session_id=338 AND item_in_session=4'''

In [11]:
#Add in the SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query_1)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

The Primary Key for the user_events table is user_id and session_id are the partition keys and item_in_session as the clustering key, so it is possible to sort song columns by item_in_session.

In [19]:
query = "CREATE TABLE IF NOT EXISTS user_events "
query = query + '''(session_id int,
                    item_in_session int,
                    user_id int,
                    first_name text,
                    last_name text,
                    artist text,
                    song text,
                    PRIMARY KEY ((user_id, session_id), item_in_session))'''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [20]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO user_events (session_id,
                                            item_in_session,
                                            user_id,
                                            first_name,
                                            last_name,
                                            artist,
                                            song)'''
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), int(line[10]), str(line[1]), str(line[4]), str(line[0]), str(line[9])))

In [21]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query = '''SELECT artist, song, first_name, last_name
           FROM user_events
           WHERE user_id=10 AND session_id=182'''

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

The Primary Key for the music_events table is song as the partition key and user_id as the clustering key. This allows the query to use these columns as filters.

In [22]:
query = "CREATE TABLE IF NOT EXISTS music_events "
query = query + '''(user_id int,
                    first_name text,
                    last_name text,
                    song text,
                    PRIMARY KEY ((song), user_id))'''
try:
    session.execute(query)
except Exception as e:
    print(e)

In [23]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Assign the INSERT statements into the `query` variable
        query = '''INSERT INTO music_events (user_id,
                                            first_name,
                                            last_name,
                                            song)'''
        query = query + " VALUES (%s, %s, %s, %s)"
        ## Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), str(line[1]), str(line[4]), str(line[9])))

In [24]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = '''SELECT first_name, last_name
           FROM music_events
           WHERE song='All Hands Against His Own'
           '''

try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


### Drop the tables before closing out the sessions

In [25]:
## Drop the table before closing out the sessions
query = "drop table song_events"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [26]:
query = "drop table user_events"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

In [27]:
query = "drop table music_events"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [28]:
session.shutdown()
cluster.shutdown()